# General information
This code is the central structure of `The Conference Calculator`. 

The sections of the basic version of the tool are:

1. Basic inputs (starting point, location of the conference).
2. Query conference database.
3. Route calculation for different modes of transportation (car, train, plane).
4. Carbon footprint calculation for different modes of transportation (car, train, plane).
5. Importance of the conference.
6. Outputs (2D plot).

## 0. Importations

In [86]:
import inquirer as iq
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import urllib.request
import os
import geopy.distance
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import osmnx as ox
from J0ANMM import carbon_car, carbon_flight, carbon_train
import wbgapi as wb
from matplotlib import colors as mcolors
%matplotlib qt

## 1. Basic inputs
Questions asked (for now!):
1. What's your starting point?
2. Where is the conference?

In [112]:
list_of_cities = {
    'London':'St Pancras International',
    'Paris': 'Gare du Nord',
    'Brussels': 'Brussel-Centraal',
    'Amsterdam': 'Amsterdam Centraal',
    'Luxembourg': 'Luxembourg Railway Station',
    'Bern': 'Bern Railway Station',
    'Berlin':'Berlin Hauptbahnhof',
    'Copenhagen':'Copenhagen Central Station',
    'Prague': 'Prague Main Railway Station',
    'Ljubljana':'Ljubljana Train Station',
    'Vienna': 'Vienna International Busterminal (VIB)',
    'Oslo':'Oslo Central Station',
    'Bratislava': 'Bratislava Hlavna',
    'Madrid': 'Madrid Atocha',
    'Zagreb':'Zagreb Glavni Kolod Train Station',
    'Rome': 'Roma Termini',
    'Budapest': 'Budapest Keleti',
    'Warsaw': 'Warszawa Centralna railway station',
    'Stockholm':'Stockholm Central Station',
    'Sarajevo':'Sarajevo Main Railway station',
    'Lisbon':'Rossio Railway Station', 
    }

print("LIST OF AVAILABLE CITIES")
print("--------------------------------------")
for city in list_of_cities.keys(): print(city)
print("--------------------------------------")

# Departure city (starting point)
starting_point = input("Please enter your departure city: ")
if starting_point not in list_of_cities.keys(): 
    raise Exception("City not available! Enter another city.")

# Destination city (destination point)
destination_point = input("Please enter your destination city: ")
if destination_point not in list_of_cities.keys(): 
    raise Exception("City not available! Enter another city.")

# Create dictionary with both cities
journey_info = {'starting_point': starting_point, 'destination_point': destination_point}
journey_info

LIST OF AVAILABLE CITIES
--------------------------------------
London
Paris
Brussels
Amsterdam
Luxembourg
Bern
Berlin
Copenhagen
Prague
Ljubljana
Vienna
Oslo
Bratislava
Madrid
Zagreb
Rome
Budapest
Warsaw
Stockholm
Sarajevo
Lisbon
--------------------------------------


{'starting_point': 'Bern', 'destination_point': 'Rome'}

## 2. Query conference database
TODO: Add Lorenzo code

## 3. Route calculation
Output: add distance from Google Maps API to `journey_info` for car, train and plane

In [113]:
# Read databases for train and car distances
car_distances_db = pd.read_csv(os.path.join(os.getcwd(), 'databases', 'Car_db.csv'), index_col=0)
train_distances_db = pd.read_csv(os.path.join(os.getcwd(), 'databases', 'Rail_db.csv'), index_col=0)

# Use database to retrieve distance between cities
journey_info['distance_car'] = car_distances_db[journey_info['starting_point']][journey_info['destination_point']] / 1000.
journey_info['distance_train'] = train_distances_db[journey_info['starting_point']][journey_info['destination_point']] / 1000.

# Retrieve latitude and longitude of both cities
latitude_starting, longitude_starting = ox.geocode_to_gdf(journey_info['starting_point']).lat.values[0], ox.geocode_to_gdf(journey_info['starting_point']).lon.values[0]
latitude_destination, longitude_destination = ox.geocode_to_gdf(journey_info['destination_point']).lat.values[0], ox.geocode_to_gdf(journey_info['destination_point']).lon.values[0]

# Use geopy to calculate the distance between the two cities (for plane)
journey_info['distance_plane'] = geopy.distance.geodesic((latitude_starting, longitude_starting), (latitude_destination, longitude_destination)).km

## 4. `gCO2` calculation based on transport mode and distance
Output: add carbon footprint from Google Maps API to `journey_info` for car, train and plane

In [114]:
# OSMNX information
country = ox.geocode_to_gdf(journey_info['destination_point'])['display_name'].values[0].split(', ')[-1]
wb_featureset = wb.economy.info(q=country)
country_ID = wb_featureset.items[0]['id'][:2]

if country_ID == 'GB': country_ID = 'UK'

# List of transport options
list_of_transport = ['petrol_car', 'electric_car', 'train', 'plane']

for mode in list_of_transport:

    # For electric and petrol cars
    if np.char.endswith(mode, 'car'):
        class_car = carbon_car.CarbonCar()
        co2_car = class_car.calculate_co2(dist_km=journey_info['distance_{}'.format(mode.split('_')[1])], \
                                       fuel_type=mode.split('_')[0], \
                                       pax_in_car=1, \
                                       trip_type='round-trip')
        
        # Save carbon footprint in dictionary
        journey_info['carbon_footprint_{}'.format(mode)] = co2_car

    # For train
    elif mode == 'train':
        class_train = carbon_train.CarbonTrain()
        co2_train = class_train.calculate_co2(dist_km=journey_info['distance_{}'.format(mode)], \
                                              train_energy='electric', \
                                              train_country=country_ID, \
                                              trip_type='round-trip')
        
        # Save carbon footprint in dictionary
        journey_info['carbon_footprint_{}'.format(mode)] = co2_train
    
    # For plane
    elif mode == 'plane':
        class_plane = carbon_flight.CarbonFlight()
        co2_plane = class_plane.calculate_co2(dist_km=journey_info['distance_{}'.format(mode)], \
                                              pax_class='economy', \
                                              trip_type='round-trip')
        
        # Save carbon footprint in dictionary
        journey_info['carbon_footprint_{}'.format(mode)] = co2_plane


## 5. Importance of the conference
- *Are you presenting?* $\rightarrow$ talk, poster, invited talk, workshop lecture, etc.
- *How often is the conference held?* $\rightarrow$ e.g., every four years, unique conference, annually.
- *How specific is the conference to your research project?* $\rightarrow$ broad, specific, expert level?
- *Is there a relevant workshop/session you want/need to attend?*
- *Is the conference offered in hybrid mode?*
- *Is there a networking session?*
- *Are you an Early Career Researcher?*

In [115]:
class ConferenceImportance:
    def __init__(self):
        self.conference_importance = {'dictionaries': {},
                                      'information': {}}
        
        dict_ECR = {"Yes": 1.5,
                    "No": 1.}
        
        dict_presenting = { "I'm not presenting anything": 1.,
                            'I was SELECTED to present a POSTER': 4.,
                            'I was INVITED to present a POSTER': 5.,
                            'I was SELECTED to give a TALK (less than 15 minutes including questions)': 6.,
                            'I was INVITED to give a TALK (less than 15 minutes including questions)': 7.5,
                            'I was SELECTED to give a TALK (15 minutes or more including questions)': 8.,
                            'I was INVITED to give a TALK (15 minutes or more including questions)': 9.5,
                            'I was INVITED to chair a session': 5.,
                            'I was INVITED to be a panelist': 8.5,
                            'I was INVITED to give a workshop': 10.,
                            'Other': None}
    
        dict_expertise = { 'Broad (e.g., STEM conference, outreach conference)': 4.,
                            'Field-specific (e.g., physics conference)': 5.,
                            'Subfield-specific (e.g., astrophysics conference)': 6.5,
                            'Topic-specific (e.g., black hole conference)': 8.,
                            'Research-specific (e.g., black hole feedback conference)': 10.,
                            'Other': None}

        dict_workshop = {'Yes': 5., 
                         'No': 0.}

        dict_hybrid = {'Yes': True,
                       'No': False}

        dict_networking = {'Yes': True,
                           'No': False}
        
        self.conference_importance['dictionaries']['dict_ECR'] = dict_ECR
        self.conference_importance['dictionaries']['dict_presenting'] = dict_presenting
        self.conference_importance['dictionaries']['dict_expertise'] = dict_expertise
        self.conference_importance['dictionaries']['dict_workshop'] = dict_workshop
        self.conference_importance['dictionaries']['dict_hybrid'] = dict_hybrid
        self.conference_importance['dictionaries']['dict_networking'] = dict_networking

    def asking_question_from_dict(self, dict_name):

        for idx, key in enumerate(self.conference_importance['dictionaries'][dict_name].keys()):
            print(idx+1, '-', key)

        user_info = int(input("Please select an option (1-{}):".format(len(self.conference_importance['dictionaries'][dict_name].keys()))))    
        user_info_score = self.conference_importance['dictionaries'][dict_name][list(self.conference_importance['dictionaries'][dict_name].keys())[user_info-1]]   

        return user_info_score

    def ask_ECR(self):
        print('\nQUESTION: Are you an Early Career Researcher (e.g., undergraduate student, postgraduate student, postdoctoral researcher, research assistant)?')
        user_info_ECR_score = self.asking_question_from_dict('dict_ECR')
        self.conference_importance['information']['info_ECR'] = user_info_ECR_score

    def ask_presenting(self):
        print('\nQUESTION: Are you presenting at the conference?')
        user_info_presenting_score = self.asking_question_from_dict('dict_presenting')

        if user_info_presenting_score == None:
            user_info_presenting_score = int(input('Please evaluate your contribution on a scale from 1 to 10: '))

        self.conference_importance['information']['info_presenting'] = user_info_presenting_score
    
    def ask_expertise(self):
        print('\nQUESTION: How specific is the conference to your field of expertise?')
        user_info_expertise_score = self.asking_question_from_dict('dict_expertise')

        if user_info_expertise_score == None:
            user_info_expertise_score = int(input('Please evaluate the importance (in terms of expertise) from 1 to 10: '))

        self.conference_importance['information']['info_expertise'] = user_info_expertise_score
    
    def ask_workshop(self):
        print('\nQUESTION: Is there a relevant workshop/session you want/need to attend?')
        user_info_workshop_score = self.asking_question_from_dict('dict_workshop')
        self.conference_importance['information']['info_workshop'] = user_info_workshop_score
    
    def ask_hydrid(self):
        print('\nQUESTION: Is the conference offered in hybrid mode?')
        user_info_hybrid_score = self.asking_question_from_dict('dict_hybrid')
        self.conference_importance['information']['info_hybrid'] = user_info_hybrid_score
    
    def ask_networking(self):
        print('\nQUESTION: Is there a networking session?')
        user_info_networking_score = self.asking_question_from_dict('dict_networking')
        self.conference_importance['information']['info_networking'] = user_info_networking_score

    def main(self):
        self.ask_ECR()
        self.ask_presenting()    
        self.ask_expertise()
        self.ask_workshop()
        self.ask_hydrid()
        self.ask_networking()

        # Calculation of score based on networking and hybrid
        if self.conference_importance['information']['info_hybrid'] and self.conference_importance['information']['info_networking']:
            score_hybrid_networking = 4.

        elif not self.conference_importance['information']['info_hybrid'] and self.conference_importance['information']['info_networking']:
            score_hybrid_networking = 6.
        
        elif self.conference_importance['information']['info_hybrid'] and not self.conference_importance['information']['info_networking']:
            score_hybrid_networking = 1.
        
        elif not self.conference_importance['information']['info_hybrid'] and not self.conference_importance['information']['info_networking']:
            score_hybrid_networking = 3.

        # Final calculation
        self.conference_importance['final_score'] = (self.conference_importance['information']['info_presenting'] + \
                                                    self.conference_importance['information']['info_expertise'] + \
                                                    self.conference_importance['information']['info_workshop'] + \
                                                    score_hybrid_networking) * \
                                                    self.conference_importance['information']['info_ECR']

        # Maximum score 
        maximum_score = 31. * 1.5

        # Minimum score 
        minimum_score = 6.

        # Relative score
        relative_score = ((self.conference_importance['final_score'] - minimum_score) / (maximum_score - minimum_score)) * 100.

        # Save relative score
        self.conference_importance['relative_score'] = relative_score

        return self.conference_importance

conference_importance = ConferenceImportance().main()
journey_info['conference_importance'] = conference_importance['relative_score']
        


QUESTION: Are you an Early Career Researcher (e.g., undergraduate student, postgraduate student, postdoctoral researcher, research assistant)?
1 - Yes
2 - No

QUESTION: Are you presenting at the conference?
1 - I'm not presenting anything
2 - I was SELECTED to present a POSTER
3 - I was INVITED to present a POSTER
4 - I was SELECTED to give a TALK (less than 15 minutes including questions)
5 - I was INVITED to give a TALK (less than 15 minutes including questions)
6 - I was SELECTED to give a TALK (15 minutes or more including questions)
7 - I was INVITED to give a TALK (15 minutes or more including questions)
8 - I was INVITED to chair a session
9 - I was INVITED to be a panelist
10 - I was INVITED to give a workshop
11 - Other

QUESTION: How specific is the conference to your field of expertise?
1 - Broad (e.g., STEM conference, outreach conference)
2 - Field-specific (e.g., physics conference)
3 - Subfield-specific (e.g., astrophysics conference)
4 - Topic-specific (e.g., black hol

## 6. Outputs
### 6.1 UK average carbon footprint (used for plotting)

### 6.1 2D plot

In [116]:
fig, ax = plt.subplots()

def getImage(path):
   if path == 'images\\electric_car.png':
      return OffsetImage(plt.imread(path, format="png"), zoom=0.06)

   else:
      return OffsetImage(plt.imread(path, format="png"), zoom=0.15)

paths = ['images\\plane.png', 'images\\electric_car.png', 'images\\car.png','images\\train.png']

#with plt.xkcd():
average_UK = 3000 * 1000.  # gCO2 per year per person

percentages = np.array([0.001, 0.01, 0.02, 0.05, 0.075, 0.1, 0.15, 0.25, 0.35, 0.5, 0.75])
cf_percentages = average_UK * percentages

# List of colors
list_c = named_colors = list(mcolors.TABLEAU_COLORS.keys())

# Min/max for xlim
min_x = 0.5 * np.min([journey_info['carbon_footprint_{}'.format(transport)] for transport in ['plane', 'electric_car', 'petrol_car', 'train']])
max_x = 1.2 * np.max([journey_info['carbon_footprint_{}'.format(transport)] for transport in ['plane', 'electric_car', 'petrol_car', 'train']])

for idx in range(len(cf_percentages)-1):
   if idx == 0:
      ax.axvspan(0, cf_percentages[idx+1], alpha=0.2, color=list_c[idx])
      if np.mean([cf_percentages[idx], cf_percentages[idx+1]]) > min_x and np.mean([cf_percentages[idx], cf_percentages[idx+1]]) < max_x:
         ax.text(np.mean([cf_percentages[idx], cf_percentages[idx+1]]), 8, '<'+str(percentages[idx+1]*100)+"%", color=list_c[idx], ha='center', fontsize=15)
         
   else: 
      ax.axvspan(cf_percentages[idx], cf_percentages[idx+1], alpha=0.2, color=list_c[idx])
      if np.mean([cf_percentages[idx], cf_percentages[idx+1]]) > min_x and np.mean([cf_percentages[idx], cf_percentages[idx+1]]) < max_x:
         ax.text(np.mean([cf_percentages[idx], cf_percentages[idx+1]]), 8, str(percentages[idx]*100)+'–'+str(percentages[idx+1]*100)+"%", color=list_c[idx], ha='center', fontsize=15)

for idx, transport in enumerate(['plane', 'electric_car', 'petrol_car', 'train']):
   print(idx, transport)
   print(journey_info['carbon_footprint_{}'.format(transport)], journey_info['conference_importance'])
   ax.plot(journey_info['carbon_footprint_{}'.format(transport)], journey_info['conference_importance'])
   ab = AnnotationBbox(getImage(paths[idx]), (journey_info['carbon_footprint_{}'.format(transport)], journey_info['conference_importance']), frameon=False)
   ax.add_artist(ab)

# Importance cut-off
ax.axhline(60, color='k', linestyle='--')

plt.ylim(0, 100)
plt.xlim(min_x, max_x)
plt.xlabel(r"$gCO_2$ of the transportation mode", fontsize=15)
plt.ylabel("Importance of the conference (relative %)", fontsize=15)
plt.title('Conference in {} (departure from {})'.format(journey_info['destination_point'], journey_info['starting_point']), fontsize=15)
plt.show()

0 plane
215165 37.03703703703704
1 electric_car
110945 37.03703703703704
2 petrol_car
388310 37.03703703703704
3 train
64773 37.03703703703704
